# DB2-Salesforce connector: Basic user information updates

In [1]:
# Parameters
hours_range = 24*10*1 # number of hours to look back

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",#GqnyixN6W3zVDAFXk80lgMAR4
} 

In [2]:
import sys
#sys.path.append(lib_dir)

import pandas as pd
import datetime

## Obtain contacts from DB2 
that have last visit date within range of interest specified by day_range

In [3]:
# Depending on the task, use different query and cutoff

# Hourly update for new registrations
date_cutoff = (datetime.datetime.today().date() - datetime.timedelta(hours=hours_range))\
                    .strftime('%Y-%m-%d')
sql_query = "select id, name, username, block, email, sendEmail, registerDate, lastvisitDate \
                    from jos_users where registerDate >= '%s'" % date_cutoff

# display
print(sql_query)

select id, name, username, block, email, sendEmail, registerDate, lastvisitDate                     from jos_users where registerDate >= '2020-08-15'


In [9]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

# get user profile details
profile_df = pd.read_sql_query("select * from jos_user_profiles where profile_key in \
('orgtype', 'organization', 'orcid') limit 500;", engine)

pcomms = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('personalizedcommunication');",engine)

up_news = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('updates_news');",engine)


In [10]:
# display
display(df.head(1))

display(profile_df.head(2))

display(pcomms.head(2))

display(up_news.head(2))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate
0,296965,Michael Alexander Austin,austin89,0,austin89@purdue.edu,0,2020-08-15 01:46:56,2020-08-23 21:30:39


,id,user_id,profile_key,profile_value,ordering,access
0,1,15623,orgtype,university,2,5
1,2,10060,orgtype,university,2,5


,id,user_id,profile_key,profile_value
0,1524493,998,personalizedcommunication,"Yes, I would like to receive personalized comm..."
1,1524495,1000,personalizedcommunication,"Yes, I would like to receive personalized comm..."


,id,user_id,profile_key,profile_value
0,1524494,998,updates_news,"No, I do not want to receive updates and news"
1,1524496,1000,updates_news,"No, I do not want to receive updates and news"


In [7]:
## adjusting new_email dataframe for the new email optin values 
# 0 or 1
pcomms_t = pcomms[['user_id','profile_value']]
up_news_t = up_news[['user_id','profile_value']]

display(pcomms_t.head(2))
display(up_news_t.head(2))


,user_id,profile_value
0,998,"Yes, I would like to receive personalized comm..."
1,1000,"Yes, I would like to receive personalized comm..."


,user_id,profile_value
0,998,"No, I do not want to receive updates and news"
1,1000,"No, I do not want to receive updates and news"


In [8]:
## filtering pcomms_t with 0 or 1
ptt = pcomms_t['profile_value'].to_list()

for index, val in enumerate(ptt):
    val = val.split(',')[0]
    if val == 'Yes':
        ptt[index] = 1 # 1 indicates opt-in
    else:
        ptt[index] = 0 # 1 indicates opt-out

pcomms_t.loc[:,'profile_value'] = ptt


C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
## filtering up news t with 0 or 1
untt = up_news_t['profile_value'].to_list()

for index, val in enumerate(untt):
    val = val.split(',')[0]
    if val == 'Yes':
        untt[index] = 1 # 1 indicates opt-in
    else:
        untt[index] = 0 # 1 indicates opt-out

up_news_t.loc[:,'profile_value'] = untt


In [28]:
## display
display(pcomms_t.head(2))
display(pcomms_t.tail(2))
display(up_news_t.head(2))
display(up_news_t.tail(2))

,user_id,profile_value
0,998,1
1,1000,1


,user_id,profile_value
217410,297861,1
217411,297862,1


,user_id,profile_value
0,998,0
1,1000,0


,user_id,profile_value
217410,297861,1
217411,297862,1


In [29]:
## create new dataframe
new_opts = pd.DataFrame()

new_opts['nanoHUB_user_ID__c'] = pcomms_t['user_id']
new_opts['personalizedcommunication__c'] = pcomms_t['profile_value']
new_opts['updatesnews__c'] = up_news_t['profile_value']


In [30]:
display(new_opts.head(2))
print(type(new_opts))

,nanoHUB_user_ID__c,personalizedcommunication__c,updatesnews__c
0,998,1,0
1,1000,1,0


<class 'pandas.core.frame.DataFrame'>


In [39]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)


Obtained Salesforce access token ...... True


In [40]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(new_opts)


[Success] Bulk job creation successful. Job ID = 7505w00000PT6BhAAL
hello
[Success] CSV upload successful. Job ID = 7505w00000PT6BhAAL
[Success] Closing job successful. Job ID = 7505w00000PT6BhAAL


In [46]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000PT6BhAAL',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-25T02:37:31.000+0000',
 'systemModstamp': '2020-08-25T02:46:00.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 217412,
 'numberRecordsFailed': 412,
 'retries': 0,
 'totalProcessingTime': 2195622,
 'apiActiveProcessingTime': 2010273,
 'apexProcessingTime': 21403328}

In [44]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",nanoHUB_user_ID__c,personalizedcommunication__c,updatesnews__c\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","1000.0","true","false"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","1819.0","true","false"\n')


In [11]:
df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orgtype'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orgtype'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'organization'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'organization'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orcid'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orcid'})

# display
display(df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid
0,296501,Anugraha Thyagatur,anu.thyagatur,0,anu.thyagatur@gmail.com,1,2020-08-10 00:01:54,2020-08-10 00:01:54,NaN,NaN,NaN


In [12]:
# Obtain company domain information from DB2
domain_df = pd.read_sql_query("select name as domain_name, domain, industry, `size range` as size, country \
from wang159_myrmekes.companies_email_domain", engine)

In [13]:
# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first').dropna()

In [14]:
# get email domain of nanoHUB users
def get_domain(this_email):
    seg_list = this_email.split('@')
    
    if len(seg_list) == 2:
        return seg_list[1].lower()
    else:
        return None
    
df['email_host'] = df['email'].apply(get_domain)

In [15]:
display(df.head(2))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host
0,296501,Anugraha Thyagatur,anu.thyagatur,0,anu.thyagatur@gmail.com,1,2020-08-10 00:01:54,2020-08-10 00:01:54,NaN,NaN,NaN,gmail.com
1,296502,Madeline Edwards,madelineedwards379,0,madelineedwards379@gmail.com,0,2020-08-10 00:27:17,2020-08-13 19:16:28,NaN,NaN,NaN,gmail.com


In [16]:
# get domain by nanoHUB organization field
df['organization'] = df.organization.astype('str').apply(str.lower)

df.loc[(df['organization']=='nan'),'organization'] = None

In [17]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
s_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\henry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
import re

def clean_domain(this_domain_name):
    
    if not this_domain_name:
        return None
    
    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z]+", " ", this_domain_name.lower())
    name_list = this_domain_name.split(' ')
        
    # remove all stop words
    name_list = ['' if x in s_words else x for x in name_list]
    
    return set(filter(None, name_list))

In [19]:
# clean domain name
#domain_df['domain_cleaned_set'] = domain_df.domain_name.apply(clean_domain)

# clean nanohub org name
df['org_cleaned_set'] = df.organization.apply(clean_domain)

In [20]:
# hash cleaned domain name
#domain_df['domain_cleaned_hash'] = domain_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

# hash cleaned clean nanohub org name
df['org_cleaned_hash'] = df['org_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

## Collaborative filtering for commerical email hosts

In [21]:
def predict_if_commerical(this_email_host_df):
    # predict if this email host is commerical
    top_cnt = this_email_host_df['org_cleaned_hash'].value_counts()
    
    if top_cnt.shape[0] > 0:
        if (top_cnt.iloc[0]/top_cnt.sum()) < 0.2:
            return True
        else:
            return False
    
    else:
        return False
    
is_email_commerical = df[['email_host', 'org_cleaned_hash']].groupby('email_host').apply(predict_if_commerical)

In [22]:
# mark commerical email addresses
df = pd.merge(df, is_email_commerical.reset_index(name='is_email_commerical'),\
         how='left', left_on='email_host', right_on='email_host' ,)

In [23]:
# obtain domain estimation from email addresses
df = pd.merge(df, domain_df[~domain_df.domain.isin(is_email_commerical[is_email_commerical].index)]['domain'],\
         how='left', left_on='email_host', right_on='domain').rename(columns={'domain':'domain_by_email'})

# display
display(df.head(2).T)

,0,1
id,296501,296502
name,Anugraha Thyagatur,Madeline Edwards
username,anu.thyagatur,madelineedwards379
block,0,0
email,anu.thyagatur@gmail.com,madelineedwards379@gmail.com
sendEmail,1,0
registerDate,2020-08-10 00:01:54,2020-08-10 00:27:17
lastvisitDate,2020-08-10 00:01:54,2020-08-13 19:16:28
orgtype,NaN,NaN
organization,None,None


## Fuzzy derive organization from nanoHUB profiles

In [25]:
# attempt direct join by hash
domain_subset_df = domain_df[domain_df.domain.isin(df.email_host.unique())][['domain', 'domain_cleaned_hash']]
domain_subset_df = domain_subset_df[~domain_subset_df.domain.isna()&~domain_subset_df.domain_cleaned_hash.isna()]

derived_df = pd.merge(df, domain_subset_df.rename(columns={'domain':'domain_by_profile'}),\
         how='left', left_on='org_cleaned_hash', right_on='domain_cleaned_hash')\
        .drop('domain_cleaned_hash', axis=1)

# display
derived_df.sample(5).T

KeyError: "['domain_cleaned_hash'] not in index"

In [ ]:
# attempt to reduce domain_df size to speedup Jaccard calculation
from itertools import chain

all_word_set = set(chain.from_iterable(derived_df[(derived_df['domain_by_profile'].isna() & (~derived_df['organization'].isna()))]\
                    .org_cleaned_set.values))

# select the domain_df entry with words from all_word_set
domain_subset_df = domain_df[domain_df.domain_cleaned_set.notnull()]
domain_subset_df = domain_subset_df[domain_subset_df.domain_cleaned_set\
                             .apply(lambda x: True if len(x.intersection(all_word_set)) > 0 else False)]

In [ ]:
def get_org(this_df):
    # look at the cohort with this label, do they have a commonly agreed domain_by_email? If so, use it
    this_dbe_count = this_df.domain_by_email.value_counts()

    if this_dbe_count.shape[0] < 1:
        return None

    this_top_dbe_perc = this_dbe_count.iloc[0]/this_dbe_count.sum()

    # if top choice occupies majority of cases, then this cohort can be implied to this domain_by_email address
    if this_top_dbe_perc > 0.5:
        return this_dbe_count.index[0] 


derived_hash = derived_df.groupby('org_cleaned_hash').apply(get_org)

In [ ]:
derived_df = pd.merge(derived_df, derived_hash.reset_index(name='domain_by_profile_infer')\
                      , how='left', left_on='org_cleaned_hash', right_on='org_cleaned_hash')

In [ ]:
# calculate a composite domain based on domain_by_email, domain_by_profile, and domain_by_profile_infer
def get_composite_domain(this_df):

    # if domain found via profile with direct match, use it
    if this_df.domain_by_profile != '':
        return this_df.domain_by_profile
    
    # if domain found via email, use it
    if this_df.domain_by_email != '':
        return this_df.domain_by_email    
    
    # if domain found via profile but not with a direct match, use it
    if this_df.domain_by_profile_infer != '':
        return this_df.domain_by_profile_infer
    
    # else, no organization can be concluded, return none
    return None

    
derived_df['domain_final'] = derived_df[['domain_by_email', 'domain_by_profile', 'domain_by_profile_infer']].fillna('')\
                                                        .apply(get_composite_domain, axis=1)

## Update Salesforce institution 

In [ ]:
# get a unique list of organizations that appears in contact
#org_df = derived_df[['domain_name', 'domain', 'industry', 'size', 'country']].drop_duplicates().dropna()
org_df = pd.merge(derived_df[['domain_final']].drop_duplicates().dropna(), domain_df, how='inner', left_on='domain_final', right_on='domain') 
org_df = org_df[['domain_name', 'domain', 'industry', 'size', 'country']]

# display
display(org_df.head(2))

In [ ]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name'] = org_df['domain_name']

df_sf['Size_Range__c'] = org_df['size'].fillna('Unknown')
df_sf['Country__c'] = org_df['country']
df_sf['Domain__c'] = org_df['domain']
df_sf['Industry__c'] = org_df['industry']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

In [ ]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = 'organization__c'
db_s.external_id = 'Domain__c'

In [ ]:
# send data to Salesforce
db_s.send_data(df_sf)

In [ ]:
# check status
db_s.check_bulk_status()

In [ ]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

## Obtain newly updated organization IDs from Salesforce

In [ ]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

In [ ]:
# get Salesforce ID for organizations
sf_org_ID_df = db_s.query_data('SELECT Id, Domain__c FROM Lead')

# display
sf_org_ID_df.head(3)

In [ ]:
# join salesforce ID back to contact DF
df = pd.merge(sf_org_ID_df, derived_df, how='right', left_on='Domain__c', right_on='domain_final')\
            .drop('Domain__c', axis=1)\
            .rename(columns={'Id':'Salesforce_org_ID'})

In [ ]:
# display
display(df.head(2))

## Match data with Salesforce format

In [26]:
# split full name into first, middle, and last names
def split_full_name(this_name):
    this_name_list = list(filter(None, this_name.split(' ')))
    
    if len(this_name_list) == 1:
        # single word name
        return pd.Series([this_name_list[0],None, this_name_list[0]])
    
    elif len(this_name_list) > 1:
        # multi word name
        return pd.Series([this_name_list[0],' '.join(this_name_list[1:-1]),this_name_list[-1]])

In [27]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf[['firstname', 'Middle_name__c', 'lastname']] = df['name'].apply(split_full_name)
#df_sf['name'] = df['name']

df_sf['nanoHUB_user_ID__c']            = df['id']
df_sf['nanoHUB_username__c']           = df['username']
df_sf['Email']                         = df['email'].fillna('').apply(lambda x: '' if '@invalid' in x else x)

# for sendEmail: 0 = opt-out, 1 = receive email. For salesforce HasOptedOutOfEmail, it's exact opposite
df_sf['HasOptedOutOfEmail']            = df['sendEmail'].apply(lambda x: 0 if x==1 else 1)
df_sf['nanoHUB_account_BLOCKED__c']    = df['block'].fillna(0)

# solidify time-related columns from datetime to string
df_sf['nanoHUB_registration_date__c']  = df['registerDate'].dt.date.fillna('').astype('str')
df_sf['nanoHUB_last_active_date__c']   = df['lastvisitDate'].dt.date.fillna('').astype('str')

# Tableau detailed view
df_sf['Detailed_user_timeline_to_Tableau__c'] = df_sf['Email'].apply(lambda x: 'https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email='+x+'\
&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false\
&:display_count=no&:showVizHome=no#6' if x != '' else '')

# nanoHUB user profile
df_sf['nanoHUB_user_page__c'] = df_sf['nanoHUB_user_ID__c'].apply(lambda x: 'https://nanohub.org/members/%d'%x if x != '' else '')

df_sf['ORCID__c'] = df['orcid'].fillna('')
df_sf['Organization__c'] = df['organization'].fillna('')

# derived information
#df_sf['Organization_email_derived__c'] = df['Salesforce_org_ID']

# secondary account of
df_sf['Secondary_Account_of__c'] = None

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
firstname,Anugraha,Madeline
Middle_name__c,,
lastname,Thyagatur,Edwards
nanoHUB_user_ID__c,296501,296502
nanoHUB_username__c,anu.thyagatur,madelineedwards379
Email,anu.thyagatur@gmail.com,madelineedwards379@gmail.com
HasOptedOutOfEmail,0,1
nanoHUB_account_BLOCKED__c,0,0
nanoHUB_registration_date__c,2020-08-10,2020-08-10
nanoHUB_last_active_date__c,2020-08-10,2020-08-13


In [32]:
display(df_sf.tail(3))
print(df_sf.shape)

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Secondary_Account_of__c
698,Marcela,,Sabogue,297341,lamarce2016,lamarce2016@gmail.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297341,,,None
699,Giulia,,jones,297342,giuliajones32,giuliajones32@gmail.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297342,,,None
700,nario,,rossi,297343,experion,aznpjmrcbgzuyxiran@mhzayt.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297343,,,None


(701, 15)


In [29]:
## filtering out accounts that have positive block attributes
spam_rows = df_sf[df_sf['nanoHUB_account_BLOCKED__c'].astype(str).str.contains("1")]
good_rows = df_sf[~df_sf['nanoHUB_account_BLOCKED__c'].astype(str).str.contains('1')]
display(spam_rows.head(2))
display(good_rows.tail(2))

,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Secondary_Account_of__c
19,play,,Nantaw,296523,playnantaw,davidmarkk089@gmail.com,0,1,2020-08-10,2020-08-10,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/296523,,,None
21,guszet,,guszett,296525,guszett,guszett@gmail.com,1,1,2020-08-10,2020-08-10,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/296525,,,None


,firstname,Middle_name__c,lastname,nanoHUB_user_ID__c,nanoHUB_username__c,Email,HasOptedOutOfEmail,nanoHUB_account_BLOCKED__c,nanoHUB_registration_date__c,nanoHUB_last_active_date__c,Detailed_user_timeline_to_Tableau__c,nanoHUB_user_page__c,ORCID__c,Organization__c,Secondary_Account_of__c
699,Giulia,,jones,297342,giuliajones32,giuliajones32@gmail.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297342,,,None
700,nario,,rossi,297343,experion,aznpjmrcbgzuyxiran@mhzayt.com,1,0,2020-08-20,2020-08-20,https://tableauqa.itap.purdue.edu/views/profil...,https://nanohub.org/members/297343,,,None


## To Salesforce Sales Cloud CRM

In [35]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [36]:
# send data to Salesforce
db_s.send_data(good_rows)

[Success] Bulk job creation successful. Job ID = 7505w00000OpQBfAAN
hello
[Success] CSV upload successful. Job ID = 7505w00000OpQBfAAN
[Success] Closing job successful. Job ID = 7505w00000OpQBfAAN


In [45]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000OpQOYAA3',
 'operation': 'delete',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-18T21:29:25.000+0000',
 'systemModstamp': '2020-08-18T21:29:26.000+0000',
 'state': 'Failed',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0,
 'errorMessage': 'ClientInputError : LineEnding is invalid on user data. Current LineEnding setting is LF'}

In [38]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


## deleting records that have spam flags

In [100]:
%load_ext autoreload
%autoreload 2
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
#db_s.external_id = external_id

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Obtained Salesforce access token ...... True


In [101]:
# extract only the ids from spam_rows
spam_ids = spam_rows['nanoHUB_user_ID__c']

display(spam_ids.head(2))

11    296432
14    296435
Name: nanoHUB_user_ID__c, dtype: int64

In [98]:
# send data to Salesforce
db_s.delete_data(spam_ids)

[Success] Bulk job creation successful. Job ID = 7505w00000OpQktAAF
hello


C:\Users\henry\Desktop\PhD Research\Nanohub RA\nanohub_salesforce_integ-master\salesforce\DB2SalesforceAPI.py:212: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  bulk_csv = bytes(df_sf.to_csv(index=False,line_terminator='\n'), 'utf-8').decode('utf-8','ignore').encode("utf-8")


[Success] CSV upload successful. Job ID = 7505w00000OpQktAAF
[Success] Closing job successful. Job ID = 7505w00000OpQktAAF


In [99]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000OpQktAAF',
 'operation': 'delete',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-18T21:52:46.000+0000',
 'systemModstamp': '2020-08-18T21:52:48.000+0000',
 'state': 'Failed',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0,
 'errorMessage': "InvalidBatch : InvalidBatch : The 'delete' batch must contain only ids"}

In [74]:
db_s.check_bulk_failed_results()

'[{"message":"An unexpected error occurred. Please include this ErrorId if you contact support: 1803954357-38396 (-28018514)","errorCode":"UNKNOWN_EXCEPTION"}]'